In [37]:
import json
import pandas as pd
import numpy as np
# !pip3 install nltk
# nltk.download('word2vec_sample')
# !pip3 install gensim 
# !pip3 install -U scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler, SMOTE
import collections
import math
import os, sys, pickle, urllib.request
import os.path as op
from collections import Counter
from tqdm.notebook import tqdm
import gensim

from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import spacy

In [38]:
def evaluate_metrics(yt, yp):
    results_pos = {}
    results_pos['accuracy'] = accuracy_score(yt, yp)
    precision, recall, f_beta, _ = precision_recall_fscore_support(yt, yp, average='weighted')
    results_pos['recall'] = recall
    results_pos['precision'] = precision
    results_pos['f1score'] = f_beta
    return results_pos

In [39]:
# Opening JSON file
f = open('/home/marios/data/dw-project-data/CMS_2010_to_June_2022_ENGLISH.json')
# returns JSON object as
# a dictionary
df = json.load(f)
df = pd.DataFrame.from_dict(df)

In [40]:
df = df[['id', 'keywordStrings', 'thematicFocusCategory', 'lastModifiedDate']]
df = df.dropna()
df['thematicFocusCategory'] = df['thematicFocusCategory'].apply(lambda x:x['name'] if x is not None else x)
df['thematicFocusCategory'] = df['thematicFocusCategory'].astype(str)
df = df[df['thematicFocusCategory']!=None]

#replacee secondary 
#clean category from secondary categories 
children_dict = {'Architecture':'Culture', 'Design':'Culture', 'Film':'Culture', 'Arts':'Culture', 
                 'Literature':'Culture', 'Music':'Culture', 'Dance':'Culture', 'Theater':'Culture',
                   'Climate':'Nature and Environment',
                  'Conflicts':'Politics', 'Terrorism':'Politics', 
                  'Corruption':'Law and Justice', 'Crime':'Law and Justice', 'Rule of Law':'Law and Justice',
                    'Press Freedom':'Law and Justice', 
                  'Diversity':'Human Rights', 'Freedom of Speech':'Human Rights', 'Equality':'Human Rights', 
                'Soccer': 'Sports',
                    'Trade':'Business', 'Globalization':'Business', 'Food Security':'Business'
}

secondary_cts = [val for val in children_dict.keys()]

df['thematicFocusCategory'] = df['thematicFocusCategory'].apply(lambda x: children_dict[x] if x in secondary_cts else x)


In [4]:
# let's load a pre-trained word2vec model from google
GoogleModel = gensim.models.KeyedVectors.load_word2vec_format('/home/marios/local_data_s2ds/GoogleNews-vectors-negative300.bin',
                                                               binary=True,)

In [41]:
#drop categories which are rare
elements_count = collections.Counter(df.thematicFocusCategory)
# printing the element and the frequency
for key, value in elements_count.items():
    if value <100:
        print(f"{key}: {value}")
        df = df[df.thematicFocusCategory != key]


Learning German: 75
Offbeat: 38
Innovation: 5


In [6]:
df

,id,keywordStrings,thematicFocusCategory,lastModifiedDate
8762,17392170,"[Africalink, Top Story, Africa on the Move, Ma...",History,2014-01-28T14:04:39.000Z
31542,19570803,"[Commerzbank, job cuts, administration, retail...",Business,2016-09-24T10:30:24.132Z
31543,35880029,"[Moody's, Turkey, ratings agency, junk status]",Business,2016-09-24T10:31:32.414Z
31544,35880177,"[Syria, Aleppo, war crimes, water, UNICEF]",Politics,2016-09-24T11:01:49.057Z
31545,35880824,"[Conflict Zone, Talk, link]",Politics,2016-09-24T11:54:44.938Z
...,...,...,...,...
175654,62320563,"[Turkey, Recep Tayipp Erdogan, Finland Sweden,...",Politics,2022-07-01T04:58:32.754Z
175655,62293434,"[pollution, gold mine, Turkey, Anagold, cyanide]",Nature and Environment,2022-06-30T19:23:57.846Z
175656,62320176,"[war, Ukraine, Russia, Russian attack, soldier...",Politics,2022-06-30T20:00:16.263Z
175657,62321333,"[France, vegetarian, steak, sausage]",Business,2022-07-02T10:16:37.968Z


In [7]:
corpus = [l for l in df['keywordStrings'].apply(lambda x: ', '.join(x))]

In [8]:
corpus

['Africalink, Top Story, Africa on the Move, Making a Difference, Behind the Headlines, Your Say, Crossroads Generation',
 'Commerzbank, job cuts, administration, retail banking',
 "Moody's, Turkey, ratings agency, junk status",
 'Syria, Aleppo, war crimes, water, UNICEF',
 'Conflict Zone, Talk, link',
 'High Five, 5 facts, Oktoberfest, dirndl, beer, beer tent',
 'Berlin, restaurant, Restlos Glücklich, surplus food, lifestyle',
 'Volte-Face, Oliver Curtis, landmarks, Photogram, photographer, lifestyle',
 'dachshund, trend dog, wiener dog, sausage dog, lifestyle',
 'Photo action, viewers’ entries, Euromaxx, photographs',
 'Zaz, singer, chanson, Je veux, lifestyle',
 'dachshund, trend dog, fashionable, Oliver Curtis, landmarks, photography, Oktoberfest, lifestyle',
 'Charlotte, North Carolina, police shooting, deadly, riots, black man, video',
 'DW Akademie, Pakistan, University of Peshawar, trauma support',
 'photography, arts, YouTube, digital culture',
 'museum, history, Washington, U

In [10]:
# Count vectorization of text
# Creating the vectorizer
vectorizer = CountVectorizer(stop_words='english')
# Converting the text to numeric data
X = vectorizer.fit_transform(corpus) 
# Preparing Data frame For machine learning
# Priority column acts as a target variable and other columns as predictors
CountVectorizedData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
CountVectorizedData['cleanFocusCategory']=df['thematicFocusCategory'].values
print(CountVectorizedData.shape)
CountVectorizedData.head()

(79795, 38605)


/home/marios/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,000,007,01,03,04,05,08,0rg,10,100,...,øystein,út,überall,ünal,ünker,ľudmila,şehriban,štefániková,żurek,cleanFocusCategory
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,History
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Business
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Business
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Politics
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Politics


In [11]:
WordsVocab=CountVectorizedData.columns[:-1]
len(WordsVocab)

38604

In [12]:
def FunctionText2Vec(inpTextData):
    # Converting the text to numeric data
    X = vectorizer.transform(inpTextData)
    CountVecData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
    # Creating empty dataframe to hold sentences
    W2Vec_Data=pd.DataFrame()
    # Looping through each row for the data
    for i in range(CountVecData.shape[0]):
        # initiating a sentence with all zeros
        Sentence = np.zeros(300)
        # Looping thru each word in the sentence and if its present in 
        # the Word2Vec model then storing its vector
        for word in WordsVocab[CountVecData.iloc[i , :]>=1]:
            #print(word)
            if word in GoogleModel.key_to_index.keys():    
                Sentence=Sentence+GoogleModel[word]
        # Appending the sentence to the dataframe
        W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
    return(W2Vec_Data)

In [13]:
W2Vec_Data=FunctionText2Vec(corpus)

/home/marios/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
# save the embeddings for future use 
np.save('/home/marios/local_data_s2ds/w2v_data_ALL_dirty.npy', W2Vec_Data)

In [15]:
W2Vec_Data

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.299662,0.283447,-0.123535,0.349365,-0.561035,-0.381592,0.598633,-1.204590,1.055176,1.244751,...,-0.342041,0.004211,-0.700806,-0.354126,-0.878662,-0.905640,-0.141602,-0.482178,0.404663,0.433105
0,0.200317,0.228638,-0.694740,0.358398,-0.019562,-0.473206,-0.455322,-0.468185,0.352570,0.208008,...,-1.140625,-0.106201,-0.200439,-0.055298,0.016968,0.594482,0.263367,-0.120850,0.468994,-0.409668
0,0.110260,-0.127075,-0.201050,0.694519,-0.411407,0.086578,0.196777,-0.756042,0.694824,0.215088,...,-0.653320,1.164688,-0.711182,0.079590,-0.189392,0.060669,0.704670,0.462891,-0.134277,0.358215
0,0.166748,0.611023,0.495667,0.155273,-0.469604,0.094238,0.381165,-0.810547,0.070190,0.612061,...,-0.222412,-0.231689,-0.383667,0.481567,-0.483643,0.057495,-0.515137,-0.443268,0.560913,0.461304
0,0.175186,-0.080078,0.268250,0.485107,-0.498779,0.637207,0.168701,-0.880371,0.427612,0.146729,...,-0.194092,-0.191406,-0.310791,-0.050293,0.201172,-0.698975,-0.377197,-1.096680,0.198547,-0.334229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-0.558105,-0.277893,0.227051,1.164795,-0.611694,0.069397,-0.253967,-0.596313,-0.141846,0.503662,...,-0.025696,0.413116,-0.407532,0.545021,0.016113,-0.062225,-0.192871,-0.139160,0.127441,0.958252
0,-0.479492,0.273071,0.307983,0.350830,-0.109985,0.210876,-0.412415,-0.345093,0.355957,0.735840,...,-0.376801,-0.797119,-0.432373,0.072754,-0.192749,-0.009094,0.237793,0.193115,0.903320,-0.124512
0,0.850372,2.286682,1.087769,2.325928,-0.659546,-0.229584,0.580933,-2.461914,0.814453,1.526855,...,-1.459595,-0.080688,-1.325317,0.553101,-0.711975,-0.254150,-2.881348,-0.745838,0.068726,1.340698
0,-0.750000,-0.212891,0.513916,1.635742,-0.263672,0.630371,0.060059,-0.555664,-0.221313,0.965996,...,0.073975,-0.083008,-0.882568,1.335449,0.618164,-0.389648,-0.002930,0.373047,0.103271,-0.008972


In [43]:

# Adding the target variable
W2Vec_Data.reset_index(inplace=True, drop=True)
W2Vec_Data['cleanFocusCategory']=CountVectorizedData['cleanFocusCategory']
 
# Assigning to DataForML variable
DataForML=W2Vec_Data
DataForML.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,cleanFocusCategory
0,0.299662,0.283447,-0.123535,0.349365,-0.561035,-0.381592,0.598633,-1.204590,1.055176,1.244751,...,0.004211,-0.700806,-0.354126,-0.878662,-0.905640,-0.141602,-0.482178,0.404663,0.433105,History
1,0.200317,0.228638,-0.694740,0.358398,-0.019562,-0.473206,-0.455322,-0.468185,0.352570,0.208008,...,-0.106201,-0.200439,-0.055298,0.016968,0.594482,0.263367,-0.120850,0.468994,-0.409668,Business
2,0.110260,-0.127075,-0.201050,0.694519,-0.411407,0.086578,0.196777,-0.756042,0.694824,0.215088,...,1.164688,-0.711182,0.079590,-0.189392,0.060669,0.704670,0.462891,-0.134277,0.358215,Business
3,0.166748,0.611023,0.495667,0.155273,-0.469604,0.094238,0.381165,-0.810547,0.070190,0.612061,...,-0.231689,-0.383667,0.481567,-0.483643,0.057495,-0.515137,-0.443268,0.560913,0.461304,Politics
4,0.175186,-0.080078,0.268250,0.485107,-0.498779,0.637207,0.168701,-0.880371,0.427612,0.146729,...,-0.191406,-0.310791,-0.050293,0.201172,-0.698975,-0.377197,-1.096680,0.198547,-0.334229,Politics


In [44]:
# let's code the y columnns
# let's changee names to numbers 
DataForML.cleanFocusCategory  = DataForML.cleanFocusCategory.astype(str)
primary_categories = np.unique(DataForML.cleanFocusCategory).tolist()

primary_categories = sorted(primary_categories)
prim_cat_dict = {}
for i, cat in enumerate(primary_categories):
    prim_cat_dict[cat] = i+1

DataForML.cleanFocusCategory = DataForML.cleanFocusCategory.apply(lambda x: prim_cat_dict[x] if x in prim_cat_dict else x)
DataForML.cleanFocusCategory  = DataForML.cleanFocusCategory.astype(int)
np.unique(DataForML.cleanFocusCategory)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21])

In [45]:

# Separate Target Variable and Predictor Variables
TargetVariable=DataForML.columns[-1]
Predictors=DataForML.columns[:-1]
X=DataForML[Predictors].values
y=DataForML[TargetVariable].values



In [46]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler
PredictorScaler=MinMaxScaler()
# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
# Generating the standardized values of X
X=PredictorScalerFit.transform(X)
 
# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0, stratify=y)

In [47]:
#let's balance classes
smote_sampler = SMOTE(random_state = 5)
X_smo, y_smo = smote_sampler.fit_resample(X_train, y_train)


elements_count = collections.Counter(y_smo)

for key, value in elements_count.items():
    print(f"{key}: {value}")

4: 20314
19: 20314
15: 20314
10: 20314
14: 20314
1: 20314
18: 20314
16: 20314
21: 20314
3: 20314
11: 20314
2: 20314
12: 20314
7: 20314
17: 20314
9: 20314
6: 20314
8: 20314
20: 20314
5: 20314
13: 20314


In [49]:
#let's try random forest on this data

model = RandomForestClassifier(max_depth= 10, max_features = 'auto', n_estimators= 20)
model.fit(X_smo, y_smo)
preds = model.predict(X_test)
evaluate_metrics(y_test, preds)

{'accuracy': 0.4758288079595944,
 'recall': 0.4758288079595944,
 'precision': 0.6357711208079646,
 'f1score': 0.5045737490839893}

In [52]:
params_grid = {
    'C': [25, 50, 150],
    'kernel': ['poly', 'rbf', 'sigmoid']
}
model = SVC()

# Define a GridSearchCV to search the best parameters
grid_search_balanced = GridSearchCV(estimator = model, 
                           param_grid = params_grid, 
                           scoring='f1',
                           cv = 5, verbose = 1)
# Search the best parameters with training data
model_fit_balanced = grid_search_balanced.fit(X_smo, y_smo)
best_params_balanced = grid_search_balanced.best_params_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


: 

: 

In [ ]:

# Define a GridSearchCV to search the best parameters
grid_search = GridSearchCV(estimator = model, 
                           param_grid = params_grid, 
                           scoring='f1',
                           cv = 5, verbose = 1)
# Search the best parameters with training data
model_fit = grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

In [51]:
#find the best model from balanced, unbalanced, cross-validation or whatever, based on f1 score.
# then create heatmap to see where it's losing points
#then retest on google keywords? see whow it does there....
#then find some good examples and pair up with the pipeline 2 in order to showcase the capabilities of the work! =) 

In [34]:
model = SVC(C=10, kernel='rbf')
model.fit(X_train, y_train)
preds = model.predict(X_test)
evaluate_metrics(y_test, preds)

{'accuracy': 0.7273383207382372,
 'recall': 0.7273383207382372,
 'precision': 0.7056318393063219,
 'f1score': 0.709761385163214}